# Data Analysis -  ERP data

#### Objectifs du projet:

- 1) Réaiser une jointure de 3 tables (ERP + LIAISON + WEB)
- 2) Calculer le chiffre d'affaire par produit des ventes en ligne
- 3) Calculer de chiffre d'affaire total des ventes en ligne
- 4) Analyse d'outliers de la colonne 'Price'
   - A) Lister les outliers
   - B) Faire une répresentation graphique des outliers

#### Libraries et Settings

In [8]:
# Importing Libraries
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt



In [9]:
# Settings
pd.set_option('display.float_format', lambda x: f'{x:,.1f}') # Pour ne pas afficher montant en scientific notation
plt.rc('figure', figsize=(15, 8)) # taille du graphique

#### Data preparation

In [10]:
# Reading datasets
!pip install openpyxl
erp = pd.read_excel('../input/erp-analysis/erp.xlsx')
liaison = pd.read_excel('../input/erp-analysis/liaison.xlsx')
web = pd.read_excel('../input/erp-analysis/web.xlsx')


In [11]:
erp.info()
erp

In [12]:
liaison.info()
liaison

In [13]:
web.info()
web

In [14]:
# Valeurs manquantes ERP
null_erp = erp.isna().sum().sort_values(ascending=False) / len(erp)*100
null_erp

In [15]:
# Valeurs manquantes LIAISON
null_liaison = liaison.isna().sum().sort_values(ascending=False) / len(liaison)*100
null_liaison

In [16]:
# Valeurs manquantes WEB
null_web = web.isna().sum().sort_values(ascending=False) / len(web)*100
null_web

In [17]:
# Supprimer les colonnes avec qui n'ont que de valeurs nulls de la table WEB
web = web.drop(columns=['tax_class', 'post_content_filtered', 'post_content', 'post_password']) # drop colonnes avec null
web.columns = web.columns.str.replace('sku', 'id_web') # changer nom de la colonne
web

In [18]:
# Check for duplicates dans la table WEB
web.duplicated().value_counts()

In [19]:
# Supprimer valeurs dupliqués de la table WEB
web = web.drop_duplicates()
web

### 1) Jointure des tables WEB + ERP + LIAISON  

In [20]:
# Merge ERP + LIAISON
erp_merged = erp.merge(liaison, on='product_id', how= 'inner')
erp_merged

In [21]:
# Merge ERP_MERGED + WEB
all_merged = web.merge(erp_merged, how= 'right', on='id_web')
all_merged 

In [22]:
# Checking for duplicates
duplicates = all_merged.duplicated().value_counts()
duplicates


### 2) Chiffre d’affaires par produit

In [23]:
# Remplacer valeurs nulls par Zéros dans les colonnes 'total_sales' et 'price'
all_merged['total_sales'] = all_merged['total_sales'].fillna(0)
all_merged['price'] = all_merged['price'].fillna(0)

In [24]:
# Calcul du Chiffre d'affaire
chiffre_affaire_prod_1 = all_merged[all_merged["onsale_web"]==1] # selectionner seulement les ventes web
chiffre_affaire_prod = chiffre_affaire_prod_1[chiffre_affaire_prod_1["post_type"]=='product'] # selectionner seulement les lignes produits et pas images / ces deux lignes contiennent les mêmes infos
chiffre_affaire_prod['chiffre_affaire'] = chiffre_affaire_prod['total_sales']*chiffre_affaire_prod['price'] # créer colonne avec le calcul du chiffre d'affaire par produit
chiffre_affaire_prod = chiffre_affaire_prod[['id_web','product_id','total_sales', 'price', 'chiffre_affaire']] # sélectionner colonnes pertinantes
chiffre_affaire_prod = chiffre_affaire_prod.sort_values(by='chiffre_affaire', ascending=False) # ordonner la table par ordre décroissante de la colonne 'chiffre d'affaire'
chiffre_affaire_prod = chiffre_affaire_prod.reset_index(drop=True) # reset index de la table
chiffre_affaire_prod

In [25]:
# checking for duplicates
duplicates = chiffre_affaire_prod.duplicated().value_counts()
duplicates

In [26]:
# supprimer les 3 valeurs dupliqués
chiffre_affaire_prod= chiffre_affaire_prod.drop_duplicates()
chiffre_affaire_prod

In [27]:
chiffre_affaire_prod.isna().sum() # checking for NA's

In [28]:
chiffre_affaire_prod.info() # nom et nb de colonnes et data type
chiffre_affaire_prod.describe() # statistiques des colonnes numériques

### 3) Total du chiffre d’affaires réalisé en ligne

In [29]:
# Calcul du chiffre d'affaire total réalisé en ligne
chiffre_affaire_web =  chiffre_affaire_prod['chiffre_affaire'].sum()
chiffre_affaire_web.round(1)

### 4) Outliers de la colonne 'Price'
* A) Les lister
* B) Faire une représentation graphique

In [30]:
# Statistiques de la colonne 'Prix'
chiffre_affaire_prod['price'].describe()

In [31]:
# Ordonner dataset par la colonne 'Price' en ordre croissante
chiffre_affaire_prod = chiffre_affaire_prod.sort_values(by='price', ascending=True).reset_index() 
chiffre_affaire_prod

#### A) Trouver les outliers et les lister

##### Methode 1) IQR Interquartile Range

In [32]:
# 1) Trouver les quartiles 1 et 2 
q1, q3= np.percentile(chiffre_affaire_prod['price'],[25,75]) 
q1 # valeur quartile 1

In [33]:
q3 # valeur quartile 2

In [34]:
# 2) Calcul du IQR
iqr = q3 - q1
iqr

In [35]:
# 3) La valeur 1.5 en dessous et en dessus du IQR
lower_bound = q1 -(1.5 * iqr) 
upper_bound = q3 +(1.5 * iqr)
lower_bound # en dessous de cet valeur c'est un outlier

In [36]:
upper_bound # en dessus de cet valeur c'est un outlier

In [37]:
# 4) Liste des IQR en dessus de 85.35
iqr_outliers = chiffre_affaire_prod[chiffre_affaire_prod["price"]> upper_bound]
iqr_outliers

* En utilisant cette methode on trouve 32 valeurs outliers

##### Methode 2) Z Score

In [38]:
# Fonction pour detecter outliers
outliers=[]
def detect_outlier(data):
    
    threshold=3 # 3 fois l'écart type de la moyenne
    mean_1 = np.mean(data) # calcule la moyenne
    std_1 =np.std(data) # calcule l'écart type/ standard deviation
    
    
    for i in data:
        z_score= (i - mean_1)/std_1 # calcul de Z score / Combien d'écart type 
        if np.abs(z_score) > threshold:  # si le z-score est supérieur à 3 x l'écart type de la moyenne
            outliers.append(i)
    return outliers

In [39]:
outlier_datapoints = detect_outlier(chiffre_affaire_prod['price'])
outlier_datapoints

* en utilisant cette methode on retrouve 14 outliers

In [40]:
# La même chose en utilisant scipy
outliers = chiffre_affaire_prod[(np.abs(stats.zscore(chiffre_affaire_prod['price'])) > 3)]
outliers

##### Methode 3) Quantile filter

In [41]:
# Trier les valeurs top 1%
quantile_dessus = chiffre_affaire_prod["price"].quantile(0.99) #filter le 1% des valeurs au dessus
filter = chiffre_affaire_prod[chiffre_affaire_prod["price"] > quantile_dessus]
filter

* Avec cette methode on trouve 8 outliers

##### Methode 4) Moyenne + 3 ou 5 fois le standard deviation

In [42]:
# Methode 4
std_triple = np.mean(chiffre_affaire_prod['price']) + (np.std(chiffre_affaire_prod['price']) * 3)
std_triple.round(0)

In [43]:
# Trier les valeurs au dessus de la valeur obtenu avec la methode 4
outliers_filter = chiffre_affaire_prod[chiffre_affaire_prod["price"] > std_triple]
outliers_filter

* Avec cette methode on trouve les mêmes quantités d'outliers qu'en utilisant la methode Zscore, soit 14.

#### B) Représentation graphique des outliers

##### Distribution / Histogram

In [44]:
# Histogram
plt.hist(chiffre_affaire_prod.price, bins=100)
plt.gca().set(title='Distribution Prix', xlabel= 'Prix', ylabel='Fréquence')

# Mean Line
plt.axvline(chiffre_affaire_prod['price'].mean(), color='r', linestyle='dashed',  linewidth=1)
plt.text(chiffre_affaire_prod['price'].mean(), s= 'Mean: {:.2f}'.format(chiffre_affaire_prod['price'].mean()), y= 65)

# Median Line
plt.axvline(chiffre_affaire_prod['price'].median(), color='g', linestyle='dashed',  linewidth=1)
plt.text(chiffre_affaire_prod['price'].median(), s= 'Median: {:.2f}'.format(chiffre_affaire_prod['price'].median()), y= 60)

# IQR Upper bound line
plt.axvline(upper_bound, color='g', linestyle='dashed',  linewidth=1)
plt.text(upper_bound, s= 'IQR Upper bound: {:.2f}'.format(upper_bound), y= 60)

# Zscore / Methode 4
plt.axvline(std_triple, color='r', linestyle='dashed',  linewidth=1)
plt.text(std_triple, s= 'Zscore/Methode 4: {:.2f}'.format(std_triple), y= 50)

##### Statterplot

In [45]:
# Scatterplot 
plt.scatter(chiffre_affaire_prod.price, chiffre_affaire_prod.product_id, c=chiffre_affaire_prod.price)
plt.gca().set(title='Scatter plot des Prix', xlabel= 'Prix', ylabel='Product ID')

# Mean Line
plt.axvline(chiffre_affaire_prod['price'].mean(), color='r', linestyle='dashed',  linewidth=1)
plt.text(chiffre_affaire_prod['price'].mean(), s= 'Mean: {:.2f}'.format(chiffre_affaire_prod['price'].mean()), y= 7300)

# Median Line
plt.axvline(chiffre_affaire_prod['price'].median(), color='g', linestyle='dashed',  linewidth=1)
plt.text(chiffre_affaire_prod['price'].median(), s= 'Median: {:.2f}'.format(chiffre_affaire_prod['price'].median()), y= 7000)

# IQR Upper bound line
plt.axvline(upper_bound, color='g', linestyle='dashed',  linewidth=1)
plt.text(upper_bound, s= 'IQR Upper bound: {:.2f}'.format(upper_bound), y= 7200)

# Zscore / Methode 4
plt.axvline(std_triple, color='r', linestyle='dashed',  linewidth=1)
plt.text(std_triple, s= 'Zscore/Methode 4: {:.2f}'.format(std_triple), y= 7000)

##### Plot

In [46]:
plt.plot(chiffre_affaire_prod['price'], 'o')
plt.gca().set(title='Plot des Prix', xlabel= 'Index', ylabel='Prix')

# Mean line
plt.axhline(chiffre_affaire_prod['price'].mean(), color='r', linestyle='dashed',  linewidth=1)
plt.text(chiffre_affaire_prod['price'].mean(), s= 'Mean: {:.2f}'.format(chiffre_affaire_prod['price'].mean()), y= 32.74)

# Median line
plt.axhline(chiffre_affaire_prod['price'].median(), color='g', linestyle='dashed',  linewidth=1)
plt.text(chiffre_affaire_prod['price'].median(), s= 'Median: {:.2f}'.format(chiffre_affaire_prod['price'].median()), y= 23.70)

# IQR Upper bound line
plt.axhline(upper_bound, color='g', linestyle='dashed',  linewidth=1)
plt.text(upper_bound, s= 'IQR Upper bound: {:.2f}'.format(upper_bound), y= 85.35)

# Zscore / Methode 4
plt.axhline(std_triple, color='g', linestyle='dashed',  linewidth=1)
plt.text(std_triple, s= 'Zscore/Methode 4: {:.2f}'.format(std_triple), y= 117)


##### Boxplot

In [47]:
# Boxplot
plt.boxplot(chiffre_affaire_prod.price, vert=False) # vert for "no vertical"
plt.gca().set(title='Outliers', xlabel= 'Prix')

# Mean Line
plt.axvline(chiffre_affaire_prod['price'].mean(), color='r', linestyle='dashed',  linewidth=1)
plt.text(chiffre_affaire_prod['price'].mean(), s= 'Mean: {:.2f}'.format(chiffre_affaire_prod['price'].mean()), y= 0.8)

# Median Line
plt.axvline(chiffre_affaire_prod['price'].median(), color='g', linestyle='dashed',  linewidth=1)
plt.text(chiffre_affaire_prod['price'].median(), s= 'Median: {:.2f}'.format(chiffre_affaire_prod['price'].median()), y= 0.6)

# IQR Upper bound line
plt.axvline(upper_bound, color='g', linestyle='dashed',  linewidth=1)
plt.text(upper_bound, s= 'IQR Upper bound: {:.2f}'.format(upper_bound), y= 0.7)

# Zscore / Methode 4
plt.axvline(std_triple, color='r', linestyle='dashed',  linewidth=1)
plt.text(std_triple, s= 'Zscore/Methode 4: {:.2f}'.format(std_triple), y= 0.8)